In [1]:
######################################################################################## IMPORTS
import nltk
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.sem.relextract import extract_rels
from os.path import isfile
import json
import ftfy
import glob
import re

######################################################################################## RUN ONLY ONCE
# download the pos tagger since its not included by default
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# enable only if fresh NLTK or lots of issues
nltk.download('all')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Sverre\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger

True

In [29]:
import spacy

In [200]:
"""
        Function to apply preprocessing on a selection of files and store it in a separate folder
        - path: root folder (preprocessed-nltk), 
        - subf desired subfolder: AA or AB, must be passed as a string (e.g. "AA") 
        - files in subfolder AA: e.g. p_n_wiki_00 
        - files in subfolder AB: e.g. p_n_wiki_00
        - start: start number file, e.g. 0-99 (no need to fill in 00, 0 is fine)
        - end: end number file, e.g. 0-99
        - the range is inclusive which means, e.g. with (0, 0) you select & pre-process file wiki_00,
        - with (32, 50) you select file wiki_32 up till wiki_50
""" 
def rel_extraction_mul_files(path, subf=None, start=None, end=None):
    if subf:
        # from file start to end
        for i in range(start, end+1):
            # to match the filename p_n_wiki_00 up till p_n_wiki_09 we add a zero in front of the number from user input if necessary
            if i < 10:
                i = "0" + str(i)
            # construct path to file name that falls within range
            f = path + subf + "/p_n_wiki_{}".format(i) 
            
            # if file exists
            if isfile(f):
                # OPEN FILE, GO THROUGH EACH SEN AND PASS THAT INTO COREF FUNCTION
                file = open(f, 'r', encoding='utf-8')
                doc = json.load(file)
                for k, v in doc.items():
                    v = ftfy.fix_text(v) # FIX ANY ENCODINGS
                    # if paragraph has more than one word
                    if len(v.split(" ")) > 1:
                        use = v
                        # tokenize sentence
                        token = word_tokenize(use)
                        # pos_tag sentence
                        pos = pos_tag(token)
                        # chunk sentence
                        chunked = ne_chunk(pos)                        
                        # (any of 'LOCATION', 'ORGANIZATION', 'PERSON', 'DURATION', 'DATE', 'CARDINAL', 'PERCENT', 'MONEY', 'MEASURE')
                        rels = extract_rels('PERSON', 'GPE', chunked, corpus='ace', pattern=re.compile(r'.*\bbear\b.*'), window = 100)
                        #TODO: FORMAT OUTPUT -> KADIR
                    break                    
    else:
        for f in glob.glob('preprocessed-nltk/*/*'):
            # OPEN EACH FILE, GO THROUGH EACH PARAGRAPH AND PASS THAT INTO COREF FUNC
            file = open(f, 'r', encoding='utf-8')
            doc = json.load(file)
            for k, v in doc.items():
                v = ftfy.fix_text(v) # FIX ANY ENCODINGS
                # if paragraph has more than one word
                if len(v.split(" ")) > 1:
                    pass #TODO: ADD SAME CODE HERE
                    #TODO: FORMAT OUTPUT -> KADIR

In [3]:
############################################################################### INITIALIZING VARIABLES
path = "preprocessed-nltk/"

In [197]:
rel_extraction_mul_files(path, "AA", 0, 9) # rel_extraction_mul_files(path)

Andrew "Tridge" Tridgell ( bear 28 February 1967 ) be an australian computer programmer . he be the author of and a contributor to the Samba file server , and co - inventor of the rsync algorithm . 
Paul Vixie be an american computer scientist whose technical contribution include Domain Name System ( DNS ) protocol design and procedure , mechanism to achieve operational robustness of dns implementation , and significant contribution to open source software principle and methodology . he also create and launch the first successful commercial anti - spam service . he author the standard UNIX system program " send " , " proxynet " , " rtty " and Vixie cron . at one point he run his own consulting business , Vixie Enterprises . 
Jan Węglarz ( bear 1947 in Poznań ) be a polish computer scientist . his current research focus on operation research . 
Adi Shamir (; bear July 6, 1952 ) be an israeli cryptographer . he be a co - inventor of the Rivest – Shamir – Adleman ( RSA ) algorithm ( along